In [25]:
from ngsolve import *
from netgen.geom2d import SplineGeometry

from netgen.occ import *
import netgen.meshing as ngm
import numpy as np
from ngsolve.webgui import *
from netgen.geom2d import unit_square

def innerL2(gfu,mesh):
    return Integrate(InnerProduct(gfu,gfu),mesh)
def L2error(gfu,exact,mesh):
    return sqrt(innerL2(gfu-exact,mesh))

draw = False


In [26]:
from netgen.meshing import MeshingParameters

# generate a triangular mesh of mesh-size 0.2

w,l= 1,1
n = 5
rect = SplineGeometry()
pnts = [(0,0), (l,0), (l,w), (0,w)]
p1,p2,p3,p4 = [rect.AppendPoint(*pnt) for pnt in pnts]
curves = [[["line",p1,p2],"bottom"],
        [["line",p2,p3],"right"],
        [["line",p3,p4],"top"],
        [["line",p4,p1],"left"]]
[rect.Append(c,bc=bc, leftdomain=1, rightdomain=0) for c,bc in curves]

mesh = Mesh(rect.GenerateMesh(maxh=0.05))


#phi = CF(sqrt(0.25-(x-0.5)**2-(y-0.5)**2) if (x-0.5)**2+(y-0.5)**2 < 0.25 else 0)
alpha = Parameter(1) # step size 
# psi initial guess
dim = 2
p = 3
# proximal galerkin iterations

fesu = H1(mesh, order=p, dirichlet="left") # L2

fesphi = L2(mesh, order=p-3) # L_inf 
fes = fesu * fesphi
print("Number of dofs: ", fes.ndof)
 #bilinear form (not linear)
u, delta = fes.TrialFunction()
v, vphi = fes.TestFunction()

# initial guesses
gfu_u = GridFunction(fesu)

psih = GridFunction(fesphi)
psih.Set(1)

uk = GridFunction(fesu)
psik = GridFunction(fesphi)

eps = 1e-4

max_iter_newton = 30
max_iter_PG = 20

tol_newton = 1e-4
tol_exit = 1e-6
wh = GridFunction(fesu)

phi = CF(x)
n = specialcf.normal(2)
for k in range(1,max_iter_PG):
    alpha.Set(2**k)
    print("Iteration of proximal galerkin: ", k)
    # assing uk, psik
    uk.vec.data = gfu_u.vec
    psik.vec.data = psih.vec
    # psik.Set(-100,definedon=mesh.Boundaries("outer"))
    print("Starting quasi newton iteration")
    for i in range(max_iter_newton):
        with TaskManager():
            wh.vec.data = gfu_u.vec
            
            # bilinear form
            a = BilinearForm(fes)
            a += alpha * grad(u) * grad(v) * dx 
            a += delta * v * dx
            a += u * vphi * dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
            # linear form rhs
            l = LinearForm(fes)
            # f = 0
            l += (-alpha * CF(12*x**2) + psik - psih)*v*dx
            l += CF(alpha * 4) * v * ds(definedon="right")
            l += (phi + exp(psih)) * vphi*dx  
            a.Assemble()
            l.Assemble()
            # solve the system
            
            gfu = GridFunction(fes)

            r = l.vec.CreateVector()
            r.data = l.vec - a.mat * gfu.vec
            gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
            gfu_u.vec.data = gfu.components[0].vec
            psih.vec.data = psih.vec + gfu.components[1].vec
            error_newton = Integrate((wh-gfu.components[0])**2, mesh)
            print("error newton",error_newton)
            if error_newton < tol_newton**2:
                break

    tol_newton = np.sqrt(Integrate((uk-gfu.components[0])**2, mesh))
        
    if tol_newton < tol_exit:
        print("Exited with exit tolerance")
        break








        



 Generate Mesh from spline geometry
Number of dofs:  5291
Iteration of proximal galerkin:  1
Starting quasi newton iteration
error newton 0.247170827419863
error newton 0.02957076379894463
error newton 0.011336959497459397
error newton 0.0010616729121061944
error newton 2.0538414197977825e-05
error newton 1.7339108300751414e-07
error newton 2.9788818402339915e-09
Iteration of proximal galerkin:  2
Starting quasi newton iteration
error newton 0.09551981142787719
Iteration of proximal galerkin:  3
Starting quasi newton iteration
error newton 0.01673802037269673
Iteration of proximal galerkin:  4
Starting quasi newton iteration
error newton 0.002861013512989573
Iteration of proximal galerkin:  5
Starting quasi newton iteration
error newton 2.0650860755258235e-05
Iteration of proximal galerkin:  6
Starting quasi newton iteration
error newton 2.2997063641623868e-07
Iteration of proximal galerkin:  7
Starting quasi newton iteration
error newton 2.0981509404182362e-08
Iteration of proximal ga

In [28]:
Draw(phi+exp(psih)-gfu_u,mesh,min=0,max=1) 


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/netgen/webgui.py:17: RuntimeWarning: overflow encountered in cast
  values = np.array(data.flatten(), dtype=dtype)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene